DOCTOR DETECT

In [19]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('best_small.pt')

# Define the input and output video paths
input_video_path = 'vids/Doc.mp4'
output_video_path = 'output_video_doc.avi'

# Open the input video
cap = cv2.VideoCapture(input_video_path)

# Get the video writer initialized to save the output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    if frame_count % 3 != 0:
        # Skip frames that are not the third frame
        continue

    # Perform object detection
    results = model(frame)
    for result in results:
        boxes = result.boxes.xyxy  # Bounding box coordinates
        confs = result.boxes.conf  # Confidence scores
        classes = result.boxes.cls  # Class labels

        for box, conf, cls in zip(boxes, confs, classes):
            x1, y1, x2, y2 = map(int, box)
            label = int(cls)
            confidence = float(conf)

            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Write the frame to the output video
    out.write(frame)

# Release everything if job is finished
cap.release()
out.release()


0: 384x640 1 doctors, 203.7ms
Speed: 17.4ms preprocess, 203.7ms inference, 14.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 doctorss, 139.0ms
Speed: 6.0ms preprocess, 139.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 doctorss, 136.3ms
Speed: 3.0ms preprocess, 136.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 doctors, 144.4ms
Speed: 2.0ms preprocess, 144.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 doctors, 136.8ms
Speed: 2.5ms preprocess, 136.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 doctors, 142.9ms
Speed: 1.0ms preprocess, 142.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 doctors, 137.5ms
Speed: 2.0ms preprocess, 137.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 doctors, 138.9ms
Speed: 2.0ms preprocess, 138.9ms inference, 1.0ms postprocess 

VITAL SIGN DETECTION - (EXPERIMENTAL)

In [46]:
import cv2
import numpy as np
import easyocr
from ultralytics import YOLO

# Initialize YOLO models
general_model = YOLO('best_Monitor.pt')  # General object detection model
vitals_model = YOLO('best_vitals-2.pt')  # Your custom model for vitals detection

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Define the threshold for low heart rate
LOW_HR_THRESHOLD = 60
HIGH_HR_THRESHOLD = 150
null_val='_?_'

def crop_roi(frame, results):
    for result in results:
        boxes = result.boxes.cpu().numpy()
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            return frame[y1:y2, x1:x2]
    return frame  # Return full frame if no detection

def detect_vitals(roi):
    results = vitals_model(roi)
    vitals_boxes = {}
    for result in results:
        boxes = result.boxes.cpu().numpy()
        for box in boxes:
            class_id = int(box.cls[0])
            class_name = vitals_model.names[class_id]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            vitals_boxes[class_name] = (x1, y1, x2, y2)
    return vitals_boxes

def process_frame(frame):
    # Use general YOLO model to detect the monitor
    general_results = general_model(frame)
    
    # Crop the frame to the detected monitor ROI
    monitor_roi = crop_roi(frame, general_results)
    
    # Detect vitals within the monitor ROI
    vitals_boxes = detect_vitals(monitor_roi)
    
    # Check if HR (heart rate) is detected
    if 'HR' in vitals_boxes:
        x1, y1, x2, y2 = vitals_boxes['HR']
        hr_roi = monitor_roi[y1:y2, x1:x2]
        
        # Use EasyOCR to extract text from the HR ROI
        grey_img = cv2.cvtColor(hr_roi, cv2.COLOR_RGB2GRAY)
        ocr_result = reader.readtext(grey_img)
        
        # Extract the first number encountered
        return extract_first_number(ocr_result), hr_roi
    
    return None, monitor_roi

def extract_first_number(ocr_result):
    for detection in ocr_result:
        text = detection[1]  # The recognized text is the second element of each detection
        try:
            return int(text)
        except ValueError:
            continue  # If it's not a number, continue to the next detection
    return None

def check_low_heart_rate(heart_rate):
    if heart_rate < LOW_HR_THRESHOLD or heart_rate > HIGH_HR_THRESHOLD or heart_rate == null_val:
        print(f"ALERT: Low heart rate detected! Current rate: {heart_rate} bpm")
        return True
    return False

def main():
    # Ask user for image file path
    image_path = 'vids/vitals-1.jpg'
    
    # Read the image
    frame = cv2.imread(image_path)
    
    if frame is None:
        print("Error opening image file")
        return
    
    heart_rate, roi = process_frame(frame)
    
    if heart_rate is not None:
        print(f"Detected Heart Rate: {heart_rate}")
        
        # Check for low heart rate and send alert if necessary
        if check_low_heart_rate(heart_rate):
            print("LOW HR ALERT!")
        
        print(f"HR: {heart_rate}")
    else:
        print("No HR detected")

if __name__ == "__main__":
    main()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 288x416 1 object, 51.4ms
Speed: 4.0ms preprocess, 51.4ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 416)

0: 416x416 1 HR, 1 SPO2, 59.2ms
Speed: 2.5ms preprocess, 59.2ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)
Detected Heart Rate: 76
HR: 76


FALL DETECTION

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import time
from collections import deque

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='tflite-model-maker-falldetect-model.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get the input shape
input_shape = input_details[0]['shape']
height = input_shape[1]
width = input_shape[2]

# Open the video stream (use 0 for webcam, or provide a video file path)
cap = cv2.VideoCapture('vids/fall.mp4')  # Use 0 for webcam, or 'path_to_your_video.mp4' for a video file

# For FPS calculation
prev_frame_time = 0
new_frame_time = 0

# Parameters for fall detection
detection_buffer = deque(maxlen=100)  # Increased buffer size
initial_fall_threshold = 0.8  # High initial threshold
min_fall_threshold = 0.7  # Minimum threshold
fall_frames_threshold = 90  # Number of frames required for fall detection
cooldown_frames = 60  # Frames to wait before allowing another detection
frames_since_last_detection = cooldown_frames  # Initialize as if we just had a detection

fall_detected = False
fall_start_time = None

# Function to calculate dynamic threshold
#def calculate_dynamic_threshold(buffer):
#    if len(buffer) < 20:
#        return initial_fall_threshold
#    sorted_probs = sorted(buffer)
#    return max(min_fall_threshold, np.mean(sorted_probs[-20:]) + 2 * np.std(sorted_probs[-20:]))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess the frame
    resized_frame = cv2.resize(frame, (width, height))
    input_data = np.expand_dims(resized_frame, axis=0).astype(np.uint8)
    
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Run inference
    interpreter.invoke()
    
    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Process the output
    fall_probability = output_data[0][0]
    detection_buffer.append(fall_probability)
    
    # Calculate dynamic threshold
    current_threshold = 230 #calculate_dynamic_threshold(detection_buffer)
    
    # Check if a fall is detected
    if frames_since_last_detection >= cooldown_frames:
        if fall_probability > current_threshold:
            fall_detected = True
            fall_start_time = time.time()
            frames_since_last_detection = 0
    else:
        frames_since_last_detection += 1
    
    # Display fall probability and threshold
    cv2.putText(frame, f"Fall Prob: {fall_probability:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(frame, f"Threshold: {current_threshold:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Display alert if fall is detected
    if fall_detected:
        color = (0, 0, 255)  # Red color for alert
        cv2.putText(frame, "FALL DETECTED!", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), color, 4)
        
        # Reset fall detection after 3 seconds
        if time.time() - fall_start_time > 3:
            fall_detected = False
    
    # Display cooldown status
    cv2.putText(frame, f"Cooldown: {frames_since_last_detection}/{cooldown_frames}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Calculate and display FPS
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    cv2.putText(frame, f"FPS: {int(fps)}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('Fall Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Chest Pain detect

In [18]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('best_chest.pt')

# Define the input and output video paths
input_video_path = 'vids\chest pain.mp4'
output_video_path = 'output_chest-pain.avi'

# Open the input video
cap = cv2.VideoCapture(input_video_path)

# Get the video writer initialized to save the output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    if frame_count % 3 != 0:
        # Skip frames that are not the third frame
        continue

    # Perform object detection
    results = model(frame)
    for result in results:
        boxes = result.boxes.xyxy  # Bounding box coordinates
        confs = result.boxes.conf  # Confidence scores
        classes = result.boxes.cls  # Class labels

        for box, conf, cls in zip(boxes, confs, classes):
            x1, y1, x2, y2 = map(int, box)
            label = int(cls)
            confidence = float(conf)

            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Write the frame to the output video
    out.write(frame)

# Release everything if job is finished
cap.release()
out.release()


0: 224x416 2 Chest-Pains, 60.6ms
Speed: 2.0ms preprocess, 60.6ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 416)

0: 224x416 1 Chest-Pain, 1 No-Pain, 37.1ms
Speed: 1.0ms preprocess, 37.1ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 416)

0: 224x416 1 Chest-Pain, 1 No-Pain, 39.0ms
Speed: 2.0ms preprocess, 39.0ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 416)

0: 224x416 3 Chest-Pains, 1 No-Pain, 36.1ms
Speed: 1.0ms preprocess, 36.1ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 416)

0: 224x416 1 Chest-Pain, 1 No-Pain, 37.1ms
Speed: 1.0ms preprocess, 37.1ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 416)

0: 224x416 2 Chest-Pains, 43.0ms
Speed: 2.0ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 416)

0: 224x416 1 Chest-Pain, 38.6ms
Speed: 1.0ms preprocess, 38.6ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 416)

0: 224x416 1 Chest-Pain, 40.1ms
Speed: 1.0